### 引用了[这里](https://github.com/pytorch/examples/tree/master/word_language_model )的部分代码

In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm
from data_utils import Dictionary, Corpus#Corpus制作单词对应索引【唯一】，通过w2id->id2w【id2w不唯一】，return ids.view(batch_size, -1)即为输入数据

## 配置GPU

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 超参数

In [3]:
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000 #要采样的单词数目
batch_size = 20
seq_length = 30
learning_rate = 0.002

## 加载宾州树库（Penn Treebank）数据集

In [4]:
corpus = Corpus()
ids = corpus.get_data('data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

## 基于RNN的语言模型

In [5]:
# 输入：x, (h, c) 我认为c也是有的，参照训练时给出的input···；x形状ids[:, i:i+seq_length]，对应下面的形状，这里seq_length既是batch_size~
# 输出：out, (h, c)， out为所有时间步的隐藏层状态，形状【num_layers, batch_size, hidden_size】
# loss： out 和 target(input的后移一个单词)，形状同上。target也可以表述为ids[:, (i+1):(i+1)+seq_length]
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        # 嵌入 单词索引 成 向量
        x = self.embed(x)
        # 前向传播LSTM
        out, (h, c) = self.lstm(x, h)
        # 将输出形状转变为 (batch_size * sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        # 解码所有时间步的隐藏层状态
        out = self.linear(out)
        return out, (h, c)

In [6]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

## 损失 和 优化 

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Truncated backpropagation截断反向传播
# detach 就是截断反向传播的梯度流
def detach(states):
    return [state.detach() for state in states]
#对于输入序列很长的问题，BPTT可能很难训练复现神经网络。除了速度之外，在很多时间步长上累积梯度可能导致值缩小到零，或者最终溢出或爆炸的值增长。
#BPTT的修改是限制在向后传递上使用的时间步数，并且实际上估计用于更新权重的梯度而不是完全计算它。
#这种变化称为截断反向传播时间或TBPTT。

## 训练模型

In [13]:
for epoch in range(num_epochs):
    # 设置隐藏层和神经元状态
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
             torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # 获得 迷你-batch 输入 和 目标
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        # 前向传播
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # 反向传播和优化
        model.zero_grad()
        loss.backward()
        clip_grad_norm(model.parameters(), 0.5)# 个人将它理解为神经网络训练时候的drop out的方法，用于解决神经网络训练过拟合的方法
        optimizer.step()
        
        step = (i+1) // seq_length
        if step % 100 == 0:
            print('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

F:\Anaconda\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch [1/5], Step[0/1549], Loss: 9.0049, Perplexity: 8142.91
Epoch [1/5], Step[100/1549], Loss: 6.0402, Perplexity: 419.98
Epoch [1/5], Step[200/1549], Loss: 5.9137, Perplexity: 370.06
Epoch [1/5], Step[300/1549], Loss: 5.7529, Perplexity: 315.10
Epoch [1/5], Step[400/1549], Loss: 5.6651, Perplexity: 288.61
Epoch [1/5], Step[500/1549], Loss: 5.1396, Perplexity: 170.64
Epoch [1/5], Step[600/1549], Loss: 5.2108, Perplexity: 183.24
Epoch [1/5], Step[700/1549], Loss: 5.3828, Perplexity: 217.64
Epoch [1/5], Step[800/1549], Loss: 5.1845, Perplexity: 178.48
Epoch [1/5], Step[900/1549], Loss: 5.0508, Perplexity: 156.15
Epoch [1/5], Step[1000/1549], Loss: 5.1414, Perplexity: 170.95
Epoch [1/5], Step[1100/1549], Loss: 5.2922, Perplexity: 198.77
Epoch [1/5], Step[1200/1549], Loss: 5.1779, Perplexity: 177.31
Epoch [1/5], Step[1300/1549], Loss: 5.0767, Perplexity: 160.25
Epoch [1/5], Step[1400/1549], Loss: 4.8640, Perplexity: 129.54
Epoch [1/5], Step[1500/1549], Loss: 5.1361, Perplexity: 170.05
Epo

## 测试模型

In [15]:
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # 初始化隐藏层和神经元
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                torch.zeros(num_layers, 1, hidden_size).to(device))
        # 随机选取一个单词的索引
        prob = torch.ones(vocab_size)
        input_ = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)
        #.multinomial作用是对input的每一行做n_samples次取值，输出的张量是每一次取值时input张量对应行的下标,每个样本被采样的概率由logits给出
        #.unsqueeze(1)作用是维度扩充
        for i in range(num_samples):
            # 前向传播RNN
            output, state = model(input_, state)
            # 从输出采样出一个单词索引(概率遵循logits)
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            #用采样的单词id填充输入，以备下一个步骤使用
            input_.fill_(word_id)
            #写入文件
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '    #EOS(end of sentence)
            f.write(word)
            
            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt


## 保存模型

In [16]:
torch.save(model.state_dict(), 'model.ckpt')